In [1]:
import logging
import re

sys.append
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scanpy as sc
from captum.attr import IntegratedGradients
from pandas import read_excel
from scipy.stats import mannwhitneyu
from sklearn.metrics import precision_recall_curve, roc_curve
import scanpypip.utils as ut

In [2]:
import scanpypip.preprocessing as pp

In [ ]:
adata = pp.read_sc_file("data/GSE117872/GSE117872_good_Data_TPM.txt")

In [ ]:
annotation = pd.read_csv('data/GSE117872/GSE117872_good_Data_cellinfo.txt',sep="\t",index_col="groups")
for item in annotation.columns:
    #adata.obs[str(item)] = annotation.loc[:,item].convert_dtypes('category').values
    adata.obs[str(item)] = annotation.loc[:,item].astype("category")

if "select_origin" in kargs:
    origin = kargs['select_origin']
    if origin!="all":
        selected=adata.obs['origin']==origin
        selected=selected.to_numpy('bool')
        adata = adata[selected, :]

sensitive = [int(row.find("Resistant")==-1) for row in adata.obs.loc[:,"cluster"]]
sens_ = ['Resistant' if (row.find("Resistant")!=-1) else 'Sensitive' for row in adata.obs.loc[:,"cluster"]]
#adata.obs.loc[adata.obs.cluster=="Holiday","cluster"] = "Sensitive"
adata.obs['sensitive'] = sensitive
adata.obs['sensitivity'] = sens_

# Cluster de score
pval = 0.05
n_genes = 50
if "pval_thres" in kargs:
    pval=kargs['pval_thres']
if "num_de" in kargs:
    n_genes = kargs['num_de']
adata = de_score(adata=adata,clustername="sensitivity",pval=pval,n=n_genes)